In [1]:
import pandas as pd

rating_history = pd.read_csv('./data/raw_data/user_rating_history.csv')
belief_data = pd.read_csv('./data/raw_data/belief_data.csv')

prior = belief_data[belief_data['isSeen'] == 0]
post = belief_data[belief_data['isSeen'] == 1]

### belief_data

In [3]:
# belief_data의 사후 평점이 user_rating_history에 전부 포함되어 있는지 확인
legacy_ratings = set(zip(rating_history['userId'], rating_history['movieId']))
post_ratings = set(zip(post['userId'], post['movieId']))

print(f'user_rating_history에 포함되어 있지 않은 사후 평점의 개수: {len(post_ratings - legacy_ratings):,}개')

user_rating_history에 포함되어 있지 않은 사후 평점의 개수: 3개


In [ ]:
# 전처리 이전 데이터 통계량
print(f'전처리 이전 전체 평점 수: {prior.shape[0]:,}개')
print(f'전처리 이전 고유 사용자 수: {prior['userId'].nunique():,}명')
print(f'전처리 이후 고유 영화 수: {prior['movieId'].nunique():,}명')

전처리 이전 전체 평점의 개수: 28,462개
전처리 이전 고유 사용자 수: 3,200명
전처리 이후 고유 영화 수: 7,521명


In [ ]:
# 중복 평점 데이터가 있다면 가장 마지막 평점을 보존
prior = prior.drop_duplicates(subset=['userId', 'movieId'], keep='last')

# 전처리 이후 데이터 통계량
print(f'전처리 이후 전체 평점 수: {prior.shape[0]:,}개')
print(f'전처리 이후 고유 사용자 수: {prior['userId'].nunique():,}명')
print(f'전처리 이후 고유 영화 수: {prior['movieId'].nunique():,}명')

전처리 이후 전체 평점의 개수: 26,020개
전처리 이후 고유 사용자 수: 3,200명
전처리 이후 고유 영화 수: 7,521명


In [6]:
# 최초 사전 평점 응답 시점
init_prior_date = prior['tstamp'].sort_values().iloc[0]
print(f'{init_prior_date} 이전의 평점 기록만을 사용하여 임베딩 벡터 계산')

2023-03-13 19:18:16 이전의 평점 기록만을 사용하여 임베딩 벡터 계산


### user_rating_history

In [7]:
# 전처리 이전 데이터 통계량
print(f'전처리 이전 전체 평점 수: {rating_history.shape[0]:,}개')
print(f'전처리 이전 고유 사용자 수: {rating_history['userId'].nunique():,}명')
print(f'전처리 이전 고유 영화 수: {rating_history['movieId'].nunique():,}명')

전처리 이전 전체 평점 수: 2,046,124개
전처리 이전 고유 사용자 수: 4,418명
전처리 이전 고유 영화 수: 85,170명


In [8]:
# 평점이 -1, NaN인 데이터 제거
rating_history = rating_history[rating_history['rating'] != -1]
rating_history = rating_history[~rating_history['rating'].isna()]

# 중복 평점 데이터가 있다면 가장 마지막 평점을 보존
rating_history = rating_history.drop_duplicates(subset=['userId', 'movieId'], keep='last', ignore_index=True)

# 최초 사전 평점 기록일(init_prior_date) 이전의 평점 데이터만을 사용
rating_history = rating_history[rating_history['tstamp'] < init_prior_date]

# 5-core 필터링
def filter_k_core(df, k=5):
    df = df.copy()

    while True:
        initial_size = len(df)

        movie_counts = df['movieId'].value_counts()
        movies_to_keep = movie_counts[movie_counts >= k].index
        df= df[df['movieId'].isin(movies_to_keep)]

        user_counts = df['userId'].value_counts()
        users_to_keep = user_counts[user_counts >= k].index
        df = df[df['userId'].isin(users_to_keep)]

        if len(df) == initial_size:
            break

    return df

core_rating_history = filter_k_core(df=rating_history)

# 전처리 이후 데이터 통계량
print(f'전처리 이후 전체 평점 수: {core_rating_history.shape[0]:,}개')
print(f'전처리 이후 고유 사용자 수: {core_rating_history['userId'].nunique():,}명')
print(f'전처리 이후 고유 영화 수: {core_rating_history['movieId'].nunique():,}개')

전처리 이후 전체 평점 수: 1,007,969개
전처리 이후 고유 사용자 수: 1,651명
전처리 이후 고유 영화 수: 17,467개


In [13]:
# 사전 평점과 사후 평점에 모두 존재하는 사용자 및 영화만을 추출
valid_users = set(core_rating_history['userId'].unique())
valid_movies = set(core_rating_history['movieId'].unique())

prior_filtered = prior[
    prior['userId'].isin(valid_users) & prior['movieId'].isin(valid_movies)
]

print(f'필터링 이후 전체 평점 수: {prior_filtered.shape[0]:,}개')
print(f'필터링 이후 고유 사용자 수: {prior_filtered['userId'].nunique():,}명')
print(f'필터링 이후 고유 영화 수: {prior_filtered['movieId'].nunique():,}개')

필터링 이후 전체 평점 수: 8,812개
필터링 이후 고유 사용자 수: 1,104명
필터링 이후 고유 영화 수: 3,525개
